In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# For optimization using pygmo
import pygmo as pg
from udp_initial_condition import udp_initial_condition

# For Plotting
import matplotlib.pyplot as plt

In [138]:
# Setup hyperparameters:
body_density = 533                  # https://sci.esa.int/web/rosetta/-/14615-comet-67p
target_altitude = 5000**2           # Target altitude squared
final_time = 20*3600.0              # Final time
start_time = 0                      # Starting time
time_step = 800.0              # Time step size for trajectory propagation.

# Bounds for initial variable domain: [r_x,r_y,r_z,v_x,v_y,v_z]
lower_bounds, upper_bounds = [-6000,-6000,-6000,0,0,0],[6000,6000,6000,2.5,2.5,2.5]

# Optimization parameters
population_size = 10
number_of_generations = 32

In [208]:
#####################################################
#  Observe:                                         #
#       This cell is for testing numerical          #
#       integration without running optimization.   #
#####################################################

import numpy as np

# Starting point
x = np.array([2105.14, 2486.95, -2175.24, 0.56, 0.19, 0.35])

# Choose algorithm: 
#  - "RKF78", "Euler" or "DP8713M"
algorithm = "RKF78"

# Setup problem class
udp = udp_initial_condition(body_density, target_altitude, final_time, start_time, time_step, lower_bounds, upper_bounds, algorithm)

# Compute trajectory
fitness_value, trajectory_info = udp.compute_trajectory(x)

# Plot optimized trajectory
trajectory_info = np.array(trajectory_info)
udp.plot_trajectory(trajectory_info[0:3, :])

plt.plot(trajectory_info[0,:],trajectory_info[2,:])


Physical dimension along x (UN-normalized):  5002.5703125 Km
Succesfull steps:  1
Current time:  10
Current step size:  10
Succesfull steps:  2
Current time:  20
Current step size:  10
Succesfull steps:  3
Current time:  30
Current step size:  10
Succesfull steps:  4
Current time:  40
Current step size:  10
Succesfull steps:  5
Current time:  50
Current step size:  10
Succesfull steps:  6
Current time:  60
Current step size:  10
Succesfull steps:  7
Current time:  70
Current step size:  10
Succesfull steps:  8
Current time:  80
Current step size:  10
Succesfull steps:  9
Current time:  90
Current step size:  10
Succesfull steps:  10
Current time:  100
Current step size:  10
Succesfull steps:  11
Current time:  110
Current step size:  10
Succesfull steps:  12
Current time:  120
Current step size:  10
Succesfull steps:  13
Current time:  130
Current step size:  10
Succesfull steps:  14
Current time:  140
Current step size:  10
Succesfull steps:  15
Current time:  150
Current step size:  

In [98]:
# Setup class
udp = udp_initial_condition(body_density, target_altitude, final_time, start_time, time_step, lower_bounds, upper_bounds)

# Setup user defined problem (udp)
prob = pg.problem(udp)
#print(prob)

# Setup optimization algorithm
algo = pg.algorithm(pg.sade(gen = number_of_generations))
pop = pg.population(prob = prob, size = population_size)

# Run optimization
algo.set_verbosity(1)
pop = algo.evolve(pop)

# Logs for output
print("Champion fitness value: ", pop.champion_f) 
print("Champion chromosome: ", pop.champion_x) 

# Retrieving champion chromosome
x = pop.champion_x

# Compute optimized trajectory
fit_val, r_store, v_store, a_store = udp.compute_trajectory(x)

# Plot optimized trajectory
udp.plot_trajectory(r_store)

plt.plot(r_store[1],r_store[2])

Physical dimension along x (UN-normalized):  5002.5703125 Km
truncation_error:  0.0024630706257084967
tau:  0.005516926086729597
it:  1
final time:  72000.0
current time:  0.09954513662842354
truncation_error:  0.0024407404798694188
tau:  0.0055169186974637725
it:  2
final time:  72000.0
current time:  0.19875033478659856
truncation_error:  0.0024240993789135246
tau:  0.005516911375244903
it:  3
final time:  72000.0
current time:  0.2977013232148378
truncation_error:  0.002411692256828246
tau:  0.005516904102949282
it:  4
final time:  72000.0
current time:  0.39646206413925256
truncation_error:  0.0024024278154684
tau:  0.0055168968678749935
it:  5
final time:  72000.0
current time:  0.49508034184128147
truncation_error:  0.002395502062427113
tau:  0.005516889660593999
it:  6
final time:  72000.0
current time:  0.5935918894083687
truncation_error:  0.002390320074482588
tau:  0.005516882474090243
it:  7
final time:  72000.0
current time:  0.6920234476381851
truncation_error:  0.00238644

KeyboardInterrupt: 